# IMDb Rating Predicition
In diesem Projekt soll herausgefunden werden, ob anhand von verschiedenen Daten von einem Film die durchschnittliche Bewertung von einem Film mit Hilfe von Machine Learning vorhersagen kann.
Dazu wird die IMDB Datenbank verwendet, die in Teilen frei zugänglich ist. Dabei waren die Hauptherausforderungen die Auswahl der Daten und die Logik hinter diesem Vorhaben.

## Die IMDb Datenbank
Ein ausgewählter Teil der IMDb Datenbank ist im Internet frei zugänglich und kann beispielsweise auf Kaggle gefunden werden.
Es handelt sich dabei um eine relationale Datenbank, welche mehrere Tabellen enthält.


In [8]:
import database_connector
db = database_connector.DataBase()
tables = db.show_tables()

for table in tables:
    field_info_text = db.get_table_field_info(table)

    print(f"Spalten der Tabelle {table}:")
    print("\n".join(field_info_text))
    print("\n")


Spalten der Tabelle most_rated_movies:
primaryTitle: VAR_STRING 
numVotes: LONG NUMERIC


Spalten der Tabelle person:
nconst: STRING NOT_NULL | PRIMARY_KEY | NO_DEFAULT
primaryName: VAR_STRING 
birthYear: LONG NUMERIC
deathYear: LONG NUMERIC
primaryProfession: VAR_STRING 
knownForTitles: VAR_STRING 


Spalten der Tabelle titleakas:
titleId: STRING NOT_NULL | NO_DEFAULT
ordering: LONG NOT_NULL | NUMERIC
title: VAR_STRING 
region: STRING 
language: STRING 
types: VAR_STRING 
attributes: VAR_STRING 
isOriginalTitle: LONG NUMERIC


Spalten der Tabelle titlebasic:
tconst: STRING NOT_NULL | PRIMARY_KEY | NO_DEFAULT
titleType: VAR_STRING 
primaryTitle: VAR_STRING 
originalTitle: VAR_STRING 
isAdult: LONG NUMERIC
startYear: LONG NUMERIC
endYear: LONG NUMERIC
runtimeMinutes: LONG NUMERIC
genres: VAR_STRING 


Spalten der Tabelle titleprincipals:
tconst: STRING NOT_NULL | PRIMARY_KEY | NO_DEFAULT
ordering: LONG NUMERIC
nconst: STRING NOT_NULL | PRIMARY_KEY | NO_DEFAULT
category: VAR_STRING 
job:

Dabei sind die Daten nicht komplett sauber, viele Einträge sind fehlerhaft oder unvollständig.
Deshalb ist es besonders wichtig, die Daten zu filtern und nur diese zu nehmen, die relevant sind.

In [9]:
db.sql("SELECT COUNT(*) FROM titlebasic")[0]

(5701079,)

Die Tabelle Titlebasic beinhaltet alle Titel und ist wie zu sehen ist 5 Millionen einträge groß

In [10]:
db.sql("SELECT COUNT(*) FROM titlebasic WHERE titleType = 'Movie'")[0]

(509332,)

Davon sind eine halbe Millionen Filme

In [11]:
db.sql("SELECT COUNT(*) FROM titlebasic inner join titleratings on titleratings.tconst = titlebasic.tconst where titleType = 'Movie'")[0]

(237738,)

Davon haben etwa 200k überhaupt eine Bewertung


In [12]:
db.sql("SELECT COUNT(*) FROM titlebasic inner join titleratings on titleratings.tconst = titlebasic.tconst where titleType = 'Movie' AND titleratings.numVotes > 50000")[0]


(3023,)

Davon haben nur 3000 über 50000 Bewertungen, ein Wert bei welchem wir festgestellt haben, dass die Bewertungen eine ausreichende Aussagekraft besitzen.
Das sind die Filme, die als "Valid_Movies" angesehen werden und in dieser view dargestellt werden.

In [13]:
db.sql("SELECT primaryTitle, startYear, runtimeMinutes, genres FROM imdb.valid_movies order by startYear DESC limit 8")

[('The Pancake Ghost', 2021, 0, 'Musical'),
 ('Mayday', 2020, 0, 'Drama'),
 ('Long Shot', 2019, 125, 'Comedy,Romance'),
 ('How to Train Your Dragon: The Hidden World',
  2019,
  104,
  'Action,Adventure,Animation'),
 ('Axegrinder 2', 2019, 80, 'Horror'),
 ('Extremely Wicked, Shockingly Evil and Vile',
  2019,
  110,
  'Biography,Crime,Drama'),
 ('Triple Frontier', 2019, 125, 'Action,Adventure,Crime'),
 ('Pokémon Detective Pikachu', 2019, 104, 'Action,Adventure,Comedy')]

Dabei sieht die Liste der best bewertetsten Filme der Ansicht auf der IMDb Website ähnlich, es scheinen also ähnliche Filter angewendet worden zu sein.

In [14]:
db.get_top_rated_movies()

[('The Shawshank Redemption', 1994, 9.3, 2159745),
 ('The Godfather', 1972, 9.2, 1482225),
 ('The Godfather: Part II', 1974, 9.0, 1032110),
 ('The Dark Knight', 2008, 9.0, 2134602),
 ('The Mountain II', 2016, 9.0, 101817),
 ('12 Angry Men', 1957, 8.9, 617607),
 ("Schindler's List", 1993, 8.9, 1120297),
 ('Pulp Fiction', 1994, 8.9, 1695159),
 ('The Lord of the Rings: The Return of the King', 2003, 8.9, 1533632),
 ('The Good, the Bad and the Ugly', 1966, 8.8, 640941),
 ('Forrest Gump', 1994, 8.8, 1662609),
 ('The Lord of the Rings: The Fellowship of the Ring', 2001, 8.8, 1548920),
 ('Fight Club', 1999, 8.8, 1725444),
 ('Inception', 2010, 8.8, 1892958),
 ('Joker', 2019, 8.8, 466912),
 ("One Flew Over the Cuckoo's Nest", 1975, 8.7, 852798),
 ('Star Wars: Episode V - The Empire Strikes Back', 1980, 8.7, 1078754),
 ('Goodfellas', 1990, 8.7, 933687),
 ('The Matrix', 1999, 8.7, 1554345),
 ('The Lord of the Rings: The Two Towers', 2002, 8.7, 1387208),
 ("It's a Wonderful Life", 1946, 8.6, 36675